In [16]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
#Read data from csv
df_orig = pd.read_csv("cosas_finales\casos_all_likereal.txt", error_bad_lines=False)
#df = df.apply(pd.to_numeric)
df_orig

C:\Users\migue\AppData\Local\Temp\ipykernel_10868\2477183066.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_orig = pd.read_csv("cosas_finales\casos_all_likereal.txt", error_bad_lines=False)


,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,35714,0.000600,178573,0.002500,357147,0.001400,0
1,35039,0.001000,175198,0.000100,350396,0.000100,0
2,34938,0.000300,174693,0.000700,349387,0.002100,0
3,34403,0.000200,172017,0.001700,344034,0.002000,0
4,39700,0.001800,198503,0.001800,397006,0.001700,0
...,...,...,...,...,...,...,...
1175995,65,0.006921,328,0.049403,657,0.064932,5
1175996,75,0.011222,375,1.771875,750,2.900309,5
1175997,69,0.000460,345,0.517402,691,0.356602,5
1175998,65,0.000769,326,0.533548,652,0.580465,5


In [3]:
#Split dataframe to make smaller in order to run quicker tests before final model
df = df_orig#.iloc[:312500, :]
#df

In [4]:
for i in range(0, len(df)-1):
  #print(str(df.tipo.values[i]) + " " + str(i))
  if df.tipo.values[i] < 0 or df.tipo.values[i] > 5:
    df = df.drop(i, axis=0)

df = df.dropna()
df

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,35714,0.000600,178573,0.002500,357147,0.001400,0
1,35039,0.001000,175198,0.000100,350396,0.000100,0
2,34938,0.000300,174693,0.000700,349387,0.002100,0
3,34403,0.000200,172017,0.001700,344034,0.002000,0
4,39700,0.001800,198503,0.001800,397006,0.001700,0
...,...,...,...,...,...,...,...
1175995,65,0.006921,328,0.049403,657,0.064932,5
1175996,75,0.011222,375,1.771875,750,2.900309,5
1175997,69,0.000460,345,0.517402,691,0.356602,5
1175998,65,0.000769,326,0.533548,652,0.580465,5


In [5]:
#Scale tamano and tiempo columns so that their values are between [0, 1]
#scaler = MinMaxScaler(feature_range=(0,1))
#df[["tamano", "tiempo","tamano.1", "tiempo.1","tamano.2", "tiempo.2"]] = scaler.fit_transform(df[["tamano", "tiempo","tamano.1", "tiempo.1","tamano.2", "tiempo.2"]])

In [6]:
#delete size data to test predictions
#df = df.drop(['tamano', 'tamano.1', 'tamano.2'], axis=1)
#df

In [7]:
#def is_float(value):
#  try:
#    float(value)
#    return True
#  except:
#    return False

#del_ids = {}
#for j in range(0, 7):
#  col = df.iloc[:,j]
#  col = np.array(col)
#  for i in range(0,len(col)-1):
#    if is_float(col[i]) == False and i in del_ids == False:
#      df[i][j] = float(df[i][j]);

#for id in del_ids:
#  df.drop(id, axis=0);

#len(del_ids)


In [8]:
#drop type column to obtain the x_train data use 6 if size included 3 if not
x_train = df.iloc[:, 0:6]
#x_train = df.iloc[:, 0:3]

#y_train = df.iloc[:,6]
x_train

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2
0,35714,0.000600,178573,0.002500,357147,0.001400
1,35039,0.001000,175198,0.000100,350396,0.000100
2,34938,0.000300,174693,0.000700,349387,0.002100
3,34403,0.000200,172017,0.001700,344034,0.002000
4,39700,0.001800,198503,0.001800,397006,0.001700
...,...,...,...,...,...,...
1175995,65,0.006921,328,0.049403,657,0.064932
1175996,75,0.011222,375,1.771875,750,2.900309
1175997,69,0.000460,345,0.517402,691,0.356602
1175998,65,0.000769,326,0.533548,652,0.580465


In [9]:
y_train = []
for i in range(0,len(df)):
  high = df.tipo.values[i]
  if high == 0:
    y_train.append([1,0,0,0,0,0])
  elif high == 1:
    y_train.append([0,1,0,0,0,0])
  elif high == 2:
    y_train.append([0,0,1,0,0,0])
  elif high == 3:
    y_train.append([0,0,0,1,0,0])
  elif high == 4:
    y_train.append([0,0,0,0,1,0])
  elif high == 5:
    y_train.append([0,0,0,0,0,1])
  else:
#    x_train.delete(i)
    print(str(i) + " " + str(df.tipo.values[i]))

len(y_train)

1176000

In [10]:
#reshape data to the type the AI model needs
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

In [11]:
#Split the data to allow train and test the model
#stratify te hace que haya una proporcion igual de cada categoria en ambos sets a ver klkpasa
x_trn, x_test, y_trn, y_test = train_test_split(x_train, y_train, stratify=y_train)

In [12]:
len(x_trn)

882000

In [13]:
real_df = pd.read_csv("cosas_finales/tiempos.csv", error_bad_lines=False)
x_train_val = real_df.iloc[:, 1:7]

y_train_val = []
for i in range(0,len(real_df)):
  high = real_df.tipo.values[i]
  if high == 0:
    y_train_val.append([1,0,0,0,0,0])
  elif high == 1:
    y_train_val.append([0,1,0,0,0,0])
  elif high == 2:
    y_train_val.append([0,0,1,0,0,0])
  elif high == 3:
    y_train_val.append([0,0,0,1,0,0])
  elif high == 4:
    y_train_val.append([0,0,0,0,1,0])
  elif high == 5:
    y_train_val.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipo.values[i]))

x_train_val = np.array(x_train_val)
y_train_val = np.array(y_train_val)

C:\Users\migue\AppData\Local\Temp\ipykernel_10868\861101657.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  real_df = pd.read_csv("cosas_finales/tiempos.csv", error_bad_lines=False)


Note, the most confusing thing here is that the shape of the input to the model is defined as an argument on the first hidden layer. This means that the line of code that adds the first layer is doing 2 things, defining the input or visible layer and the first hidden layer.

In [14]:
#Generate a sequential model with 4 layers 
#The last one has 6 nodes, one for each execution time order [cte, logn, n, nlogn, n2, n3]
#The ones in between have arbitrary values obtain through testing ( still may be swapped )

#redes GRU igual esta interesante

model = Sequential()
#model.add(no. de entradas de cada caso) 
#esto es asi, en la libreria keras, en el modelo secuencial, se define en input_shape el nº de nodos de la entrada
model.add(LSTM(200, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(200))
model.add(Dropout(0.2))
#model.add(Dense(50))
model.add(Dense(6, activation = 'softmax'))

#model.add(Dense(50, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dense(25, activation='relu'))
#model.add(Dense(5, activation='relu'))

#Compile the model previously created with the chosen parameters
#optimizer='sgd'
model.compile(optimizer='adam', loss = CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6, 200)            161600    
                                                                 
 dropout (Dropout)           (None, 6, 200)            0         
                                                                 
 lstm_1 (LSTM)               (None, 6, 200)            320800    
                                                                 
 dropout_1 (Dropout)         (None, 6, 200)            0         
                                                                 
 lstm_2 (LSTM)               (None, 6, 200)            320800    
                                                                 
 dropout_2 (Dropout)         (None, 6, 200)            0         
                                                                 
 lstm_3 (LSTM)               (None, 200)               3

In [15]:
#Train the model with the 'train' portion of the data 
#The other parameters are tuned down in order to reduce train time but should be higher
callback = EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights=True)

model.fit(x_trn, y_trn, batch_size=16, epochs=20, callbacks=[callback], validation_data=(x_train_val, y_train_val))

Epoch 1/20
  327/55125 [..............................] - ETA: 15:54 - loss: 1.3866 - accuracy: 0.3228

KeyboardInterrupt: 

In [ ]:
#Save trained model to a file
model.save("model.h5")

In [ ]:
#model = load_model("model_4pcntError.h5")
#model.summary()

In [ ]:
#Use the model to predict the test data in order to find the success percent in the test data

#x_test = np.array(x_test)
#y_test = np.array(y_test)

predictions = model.predict(x_test)
predictions

array([[5.6304762e-05, 8.6904159e-03, 5.4812384e-01, 1.6912049e-01,
        2.7400884e-01, 1.2893764e-07],
       [9.5749795e-01, 4.7819773e-04, 3.7271746e-02, 4.7475835e-03,
        4.4257745e-06, 1.0936357e-13],
       [2.3035893e-11, 9.9997473e-01, 8.5852819e-10, 2.5288287e-05,
        2.9822067e-09, 7.5839149e-14],
       ...,
       [2.5960041e-12, 5.0075852e-16, 2.0624519e-16, 6.1787288e-17,
        5.6810326e-15, 1.0000000e+00],
       [5.8301928e-09, 2.8739366e-01, 5.0021299e-06, 7.1248245e-01,
        1.1894411e-04, 3.4060667e-11],
       [1.3936925e-08, 2.0340286e-01, 8.1818922e-05, 7.4650955e-01,
        5.0005652e-02, 1.8709798e-09]], dtype=float32)

In [ ]:
#Calculate error generated in the predictions of test data
errors = predictions - y_test
errors

array([[ 5.6304762e-05,  8.6904159e-03, -4.5187616e-01,  1.6912049e-01,
         2.7400884e-01,  1.2893764e-07],
       [-4.2502046e-02,  4.7819773e-04,  3.7271746e-02,  4.7475835e-03,
         4.4257745e-06,  1.0936357e-13],
       [ 2.3035893e-11, -2.5272369e-05,  8.5852819e-10,  2.5288287e-05,
         2.9822067e-09,  7.5839149e-14],
       ...,
       [ 2.5960041e-12,  5.0075852e-16,  2.0624519e-16,  6.1787288e-17,
         5.6810326e-15,  0.0000000e+00],
       [ 5.8301928e-09,  2.8739366e-01,  5.0021299e-06, -2.8751755e-01,
         1.1894411e-04,  3.4060667e-11],
       [ 1.3936925e-08,  2.0340286e-01,  8.1818922e-05, -2.5349045e-01,
         5.0005652e-02,  1.8709798e-09]], dtype=float32)

In [ ]:
real_preds = []
for i in range(0,len(predictions)):
  p = predictions[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0,0,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1,0,0])
  elif max(p) == p[4]:
    real_preds.append([0,0,0,0,1,0])
  elif max(p) == p[5]:
    real_preds.append([0,0,0,0,0,1])
real_preds

[[0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0,

In [ ]:
#Calculate error generated in the "real"predictions of test data
errors = real_preds - y_test
errors

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [ ]:
error_count = 0
err = real_preds - y_test
for i in range(0, len(err)):
  if max(err[i]) != min(err[i]):
    error_count += 1
print(str(error_count) + " / " + str(len(real_preds)) + " => real error% of " + str(error_count/len(real_preds)*100))

62038 / 294000 => real error% of 21.101360544217687


Cappear el tiempoEjecucion maximo con double.max o similar para poder meter tamaños mas grandes al calcula tiempo con n2 y n3 (cuidado introduce mas ruido!!) **done**

Añadir otra error count aceptando como aciertos mas casos: EJ: un n2 lo llama n3 es un acierto. **idk**

Probar a sustituir los tamaños por los porcentajes de tam maximo
o probar a quitar los tamaños
=> para verificar que no aprende a predecir solo en base al tamaño **done**


Probar con casos reales del programa de alvaro. **todo**


t2 - t1 y t3 - t1 => plot

Predecir basandose en un documento con tiempos externo a los de train y test


In [ ]:
real_df = pd.read_csv("cosas_finales/tiempos.csv", error_bad_lines=False)
real_df

C:\Users\migue\AppData\Local\Temp\ipykernel_21156\2082299864.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  real_df = pd.read_csv("cosas_finales/tiempos.csv", error_bad_lines=False)


,name,N,seconds,N.1,seconds.1,N.2,seconds.2,tipo
0,niniosPrimero,10,0.001838,100000,0.033445,200000,0.065974,2
1,palmerasNieve,10000,0.004839,50000,0.010540,100000,0.016269,2
2,--erasmus,1000,0.006315,50000,0.013816,99000,0.030289,2
3,mergeSort,21474,0.018019,107374,0.047188,212600,0.095220,3
4,nosInvaden,10000,0.017800,50000,0.075275,100000,0.187217,3
5,--cazaTesoros,10,0.004479,50,0.018389,100,0.029830,4
6,insercion,2147,0.005759,10735,0.078523,21255,0.313791,4
7,tiroPalindromo,100,0.022623,500,0.034553,990,0.066085,4


In [ ]:
#real_df = real_df.drop(['N', 'N.1', 'N.2'], axis=1)
x_train = real_df.iloc[:, 1:7]
#x_train = real_df.iloc[:, 0:3]

x_train = np.array(x_train)
x_train

array([[1.00000e+01, 1.83829e-03, 1.00000e+05, 3.34450e-02, 2.00000e+05,
        6.59738e-02],
       [1.00000e+04, 4.83884e-03, 5.00000e+04, 1.05405e-02, 1.00000e+05,
        1.62693e-02],
       [1.00000e+03, 6.31468e-03, 5.00000e+04, 1.38162e-02, 9.90000e+04,
        3.02892e-02],
       [2.14740e+04, 1.80189e-02, 1.07374e+05, 4.71880e-02, 2.12600e+05,
        9.52201e-02],
       [1.00000e+04, 1.77996e-02, 5.00000e+04, 7.52747e-02, 1.00000e+05,
        1.87217e-01],
       [1.00000e+01, 4.47880e-03, 5.00000e+01, 1.83888e-02, 1.00000e+02,
        2.98302e-02],
       [2.14700e+03, 5.75876e-03, 1.07350e+04, 7.85231e-02, 2.12550e+04,
        3.13791e-01],
       [1.00000e+02, 2.26230e-02, 5.00000e+02, 3.45530e-02, 9.90000e+02,
        6.60845e-02]])

In [ ]:
y_train = []
for i in range(0,len(real_df)):
  high = real_df.tipo.values[i]
  if high == 0:
    y_train.append([1,0,0,0,0,0]);
  elif high == 1:
    y_train.append([0,1,0,0,0,0]);
  elif high == 2:
    y_train.append([0,0,1,0,0,0]);
  elif high == 3:
    y_train.append([0,0,0,1,0,0]);
  elif high == 4:
    y_train.append([0,0,0,0,1,0]);
  elif high == 5:
    y_train.append([0,0,0,0,0,1]);
  else:
    print(str(i) + " " + str(real_df.tipo.values[i]));

len(y_train)

8

In [ ]:
y_train = np.array(y_train)
y_train

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0]])

In [ ]:
real_df_preds = model.predict(x_train)
real_df_preds

array([[1.65462736e-02, 8.49182718e-03, 3.45755219e-02, 5.75419851e-02,
        1.56860754e-01, 7.25983679e-01],
       [2.54996157e-05, 1.94771476e-02, 6.70842052e-01, 1.73175484e-01,
        1.36479825e-01, 3.38998909e-08],
       [4.24418722e-05, 3.42960358e-02, 6.41040862e-01, 2.08415493e-01,
        1.16205156e-01, 1.06541087e-07],
       [3.54199226e-09, 1.87301010e-01, 3.33823882e-05, 8.11957240e-01,
        7.08289095e-04, 2.24036015e-11],
       [5.20891663e-12, 8.65556002e-02, 5.43559167e-07, 9.11338031e-01,
        2.10578181e-03, 1.87077480e-12],
       [1.39291177e-03, 3.83391208e-03, 6.89300767e-04, 2.36774571e-02,
        5.87000996e-02, 9.11706328e-01],
       [1.32281066e-08, 1.16258860e-03, 7.23384687e-07, 1.50011301e-01,
        8.48825395e-01, 2.80392829e-08],
       [5.16217528e-03, 1.07024945e-02, 1.00854263e-01, 3.11859772e-02,
        4.44976464e-02, 8.07597518e-01]], dtype=float32)

In [ ]:
err = real_df_preds - y_train
err

array([[ 1.65462736e-02,  8.49182718e-03, -9.65424478e-01,
         5.75419851e-02,  1.56860754e-01,  7.25983679e-01],
       [ 2.54996157e-05,  1.94771476e-02, -3.29157948e-01,
         1.73175484e-01,  1.36479825e-01,  3.38998909e-08],
       [ 4.24418722e-05,  3.42960358e-02, -3.58959138e-01,
         2.08415493e-01,  1.16205156e-01,  1.06541087e-07],
       [ 3.54199226e-09,  1.87301010e-01,  3.33823882e-05,
        -1.88042760e-01,  7.08289095e-04,  2.24036015e-11],
       [ 5.20891663e-12,  8.65556002e-02,  5.43559167e-07,
        -8.86619687e-02,  2.10578181e-03,  1.87077480e-12],
       [ 1.39291177e-03,  3.83391208e-03,  6.89300767e-04,
         2.36774571e-02, -9.41299900e-01,  9.11706328e-01],
       [ 1.32281066e-08,  1.16258860e-03,  7.23384687e-07,
         1.50011301e-01, -1.51174605e-01,  2.80392829e-08],
       [ 5.16217528e-03,  1.07024945e-02,  1.00854263e-01,
         3.11859772e-02, -9.55502354e-01,  8.07597518e-01]])

In [ ]:
real_preds = []
for i in range(0,len(real_df_preds)):
  p = real_df_preds[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0,0,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1,0,0])
  elif max(p) == p[4]:
    real_preds.append([0,0,0,0,1,0])
  elif max(p) == p[5]:
    real_preds.append([0,0,0,0,0,1])

real_preds

[[0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1]]

In [ ]:
err = real_preds - y_train
err

array([[ 0,  0, -1,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  1],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  1]])

añadir O(1).
t2 - t1 y t3 - t1 => plot (esto no hace falta era para el debug)

In [ ]:
def numToType( t):
  if t==0: return "CTE"
  elif t==1:return "LOG"
  elif t==2:return "N"
  elif t==3:return "NLOGN"
  elif t==4:return "N^2"
  elif t==5:return "N^3"

error_count = 0
for i in range(0, len(err)):
  if max(err[i]) != min(err[i]):
    print(str(real_df.name.values[i]) + " sol: " + numToType(real_df.tipo.values[i]))
    error_count += 1
print(str(error_count) + " / " + str(len(real_preds)) + " => real error% of " + str(error_count/len(real_preds)*100))

niniosPrimero sol: N
--cazaTesoros sol: N^2
tiroPalindromo sol: N^2
3 / 8 => real error% of 37.5


In [ ]:
#error teniendo en cuenta la aplicacion real del programa ( no todos los fallos son fallos )
#ejemplo: espero un lineal y entregan un n2, lo interpreto como n3, el resultado seria "no es lineal" y no seria un fallo
#creo que este tipo de error_count solo se puede calcular aplicando el modelo a un problema donde espero un resultado y entregan otro, no se como hacerlo aqui

y_train = []            #tipo de ejecucion real del programa
expected_train = []     #tipo de ejecucion esperado por el problema
predictions_train = []  #ordenes predecidos por la ia

correct = 0
partial_corrects = 0
errors = 0
for i in range(0, len(real_preds)):
  guess_id = np.argmax(predictions_train[i])
  answer_id = np.argmax(y_train[i])
  expected_id = np.argmax(expected_train[i])
  if guess_id == answer_id:
    correct += 1
  elif guess_id > answer_id and expected_id != answer_id:
    partial_corrects += 1
  else:
    errors += 1

print(str(errors) + " / " + str(len(real_preds)) + " => useful error% of " + str(errors/len(real_preds)*100))
print(str(correct) + " + " + str(partial_corrects) + " = " + str(correct+partial_corrects)  + " / " + str(len(real_preds)) + " => useful guesses% of " + str((correct+partial_corrects)/len(real_preds)*100))


#error_count = 0
#for i in range(0, len(real_preds)):
#  guess_id = np.argmax(real_preds[i])
#  answer_id = np.argmax(y_test[i])
#  if guess_id < answer_id:
#    error_count += 1
#print(str(error_count) + " / " + str(len(real_preds)) + " => useful error% of " + str(error_count/len(real_preds)*100))

IndexError: list index out of range